<h2><center> Welcome to the Landslide Prediction Challenge</h2></center>

A landslide is the movement of a mass of rock, debris, or earth(soil) down a slope. As a common natural hazard, it can lead to significant losses of human lives and properties.


Hong Kong, one of the hilly and densely populated cities in the world, is frequently affected by extreme rainstorms, making it highly susceptible to rain-induced natural terrain landslides

<img src = "https://drive.google.com/uc?export=view&id=1-8sSI75AG3HM89nDJEwo6_KJbAEUXS-r">

The common practice of identifying landslides is visual interpretation which, however, is labor-intensive and time-consuming.

***Thus, this hack will focus on automating the landslide identification process using artificial intelligence techniques***

This will be achieved by using high-resolution terrain information to perform the terrain-based landslide identification. Other auxiliary data such as the lithology of the surface materials and rainfall intensification factor are also provided.


Table of contents:

1. [Import relevant libraries](#Libraries)
2. [Load files](#Load)
3. [Preview files](#Preview)
4. [Data dictionary](#Dictionary)
5. [Data exploration](#Exploration)
6. [Target distribution](#Target)
7. [Outliers](#Outliers)
8. [Correlations](#Correlations)
9. [Model training](#Model)
10. [Test set predictions](#Predictions)
11. [Creating a submission file](#Submission)
12. [Tips to improve model performance](#Tips)

<a name = "Libraries"></a>
## 1. Import relevant libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report,confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

<a name = "Load"></a>
## 2. Load files

In [ ]:
!git clone https://github.com/leopoldpoldus/UNEP-STARTHACK22.git

In [ ]:
# Read files to pandas dataframes
train = pd.read_csv('/content/UNEP-STARTHACK22/Dataset/Train.csv')
test = pd.read_csv('/content/UNEP-STARTHACK22/Dataset/Test.csv')
sample_submission = pd.read_csv('/content/UNEP-STARTHACK22/Dataset/Sample submission.csv')

<a name = "Preview"></a>
## 3. Preview files

In [ ]:
# Check the first five rows of the train set
train.head()

In [ ]:
# Check the first five rows of the test set
test.head()

In [ ]:
# Check how the submission file should look like
sample_submission.head()

<a name = "Dictionary"></a>
## 4. Data Dictionary
<figure>
<img src = "https://drive.google.com/uc?export=view&id=1T_XBSH6ozmhGiDz_nL4bQvvonHUpbCfW" height = "200">
<img src = "https://drive.google.com/uc?export=view&id=13nSrrIowiFPjAgiR--Nd4cHLVwvXFaFj" height = "400">

In [ ]:
# Check shape and size of train and test set
train.shape, test.shape, sample_submission.shape

<a name = "Exploration"></a>
## 5. Data exploration

In [ ]:
# Check statistical summaries of the train set
train.describe()

 - There is a very high correlation between features extracted from the same location

In [ ]:
# Elevation correlations
plt.figure(figsize = (20, 12))
sample_elevations = ['1_elevation',	'2_elevation',	'3_elevation',	'4_elevation',	'5_elevation']
sns.pairplot(train[sample_elevations], kind="scatter", plot_kws=dict(s=80, edgecolor="white", linewidth=2.5))
plt.show()

In [ ]:
# Check statistical summaries of the test set
test.describe()

In [ ]:
# Check for any missing values
train.isnull().sum().any(), test.isnull().sum().any()

In [ ]:
# Check for duplicates
train.duplicated().any(), test.duplicated().any()

<a name = "Target"></a>
## 6. Target variable distribution

In [ ]:
# Check distribution of the target variabe
train.Label.value_counts(normalize = True)

In [ ]:
# sns.set_style('darkgrid')
# plt.figure(figsize=(7, 6))
# sns.countplot(x= train.Label)
# plt.title('Target Variable Distribution')
# plt.show()

The dataset is highly imbalanced with the majority class having 75% and the minority class 25%

Some techiques in handling class imbalance include;
 1. Using SMOTE to create synthetic data to reduce imbalanceness
 2. Undersampling the majority class
 3. Oversampling the minority class
 4. Giving more weight to minority class during modelling

<a name = "Outliers"></a>
## 7. Outliers

In [ ]:
# Exploring some features for cell 1
explore_cols =  ['1_elevation', '1_aspect', '1_slope', '1_placurv', '1_procurv', '1_lsfactor']
explore_cols

In [ ]:
# Plotting boxplots for each of the numerical columns
fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize = (20, 10))
fig.suptitle('Box plots showing outliers', y= 0.93, fontsize = 15)

for ax, data, name in zip(axes.flatten(), train, explore_cols):
  sns.boxplot(train[name], ax = ax)



 Elevation, IsFactor, Placurv, curve and slope have some outliers.
 The aspect feature has no outliers.
 
 Some of the techniques you can use to handle outliers include:
  1. Log transformations, scaling, box-cox transformations...
  2. Dropping the outliers
  3. Replacing the outliers with mean, median, mode or any other aggregates

<a name = "Correlations"></a>
## 8. Correlations

In [ ]:
# # Type of correlations 
# plt.figure(figsize = (20, 12))
# sns.pairplot(train[explore_cols], kind="scatter", plot_kws=dict(s=80, edgecolor="white", linewidth=2.5))
# plt.show()

- There is no correlation for most of the features, how can you capture this information for modelling...
- Which information can you derive from this correlations

In [ ]:
# Quantify correlations
# corr = train[explore_cols].corr()
# plt.figure(figsize = (13, 8))
# sns.heatmap(corr, cmap='RdYlGn', annot = True, center = 0)
# plt.title('Correlogram', fontsize = 15, color = 'darkgreen')
# plt.show()

 - There is a strong positive correlation of approximately 0.8 between slope and IsFactor
 - There is some negative correlation between IsFactor and placurv
 - The IsFactor variable is correlated most of the other features, why is this?

Data Preprocessing

One Hot Encode Geo Categorie -> useless

In [ ]:
plt.hist(train[train.Label == 0]['1_slope'], label = 'no_slide')
plt.hist(train[train.Label == 1]['1_slope'], label = 'land_slide')
plt.legend()

In [ ]:
plt.hist(train[train.Label == 0]['1_slope']/train[train.Label == 0]['1_twi'], label = 'no_slide')
plt.hist(train[train.Label == 1]['1_slope']/train[train.Label == 1]['1_twi'], label = 'land_slide')
plt.legend()

In [ ]:
plt.hist(train[train.Label == 0]['1_procurv'], label = 'no_slide')
plt.hist(train[train.Label == 1]['1_procurv'], label = 'land_slide')
plt.legend()

In [ ]:
plt.hist(train[train.Label == 0][[f'{i}_slope' for i in range(1, 26)]].mean(axis = 1), label = 'no_slide')
plt.hist(train[train.Label == 1][[f'{i}_slope' for i in range(1, 26)]].mean(axis = 1), label = 'land_slide')
plt.legend()

In [ ]:
plt.scatter(train[train.Label == 0]['1_slope'],
            train[train.Label == 0]['1_geology'])
            

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(handle_unknown='ignore')
# geology_features = [f'{i}_geology' for i in range(1, 26)]
# enc.fit(train[geology_features])
# new_cols = []
# for tile in range(1, 26):
#   for cat in range (1, 8):
#     new_cols.append(f'{tile}_geo_cat_{cat}')
# # print(new_cols)


# one_hot_geology_train = pd.DataFrame(enc.transform(train[geology_features]).toarray(), columns = new_cols)
# one_hot_geology_test = pd.DataFrame(enc.transform(test[geology_features]).toarray(), columns = new_cols)

# new_cols_rearanged = []
# for cat in range (1, 8):
#   for tile in range(1, 26):
#     new_cols_rearanged.append(f'{tile}_geo_cat_{cat}')

# one_hot_geology_train = one_hot_geology_train[new_cols_rearanged] 
# one_hot_geology_train['Sample_ID'] = train['Sample_ID']
# one_hot_geology_test = one_hot_geology_test[new_cols_rearanged]
# one_hot_geology_test['Sample_ID'] = test['Sample_ID']

# test_new = test.merge(one_hot_geology_test)
# test_new = test_new[list(test.columns)+list(new_cols_rearanged)]
# train_new = train.merge(one_hot_geology_train)
# train_new = train_new[list(train.columns[:-1])+list(new_cols_rearanged)+['Label']]


In [ ]:
def add_df_to_train_and_test(df_train, df_test, train, test, name):
  df_test.columns = [f'{i}_{name}' for i in range(1, 26)]
  df_train.columns = [f'{i}_{name}' for i in range(1, 26)]

  df_train['Sample_ID'] = train['Sample_ID']
  df_test['Sample_ID'] = test['Sample_ID']

  test_new = test.merge(df_test)
 
  test_new = test_new[list(test.columns)+list(df_test.columns)[:-1]] # TODO macht das was?
  # print(test_new.columns)
  train_new = train.merge(df_train)
  train_new = train_new[list(train.columns[:-1])+list(df_train.columns[:-1])+['Label']]
  # print(train_new.columns)
  return train_new, test_new


In [ ]:
def get_combined_metrics_for_tiles(cat, train, test):
  df_train = train[[f'{i}_{cat}' for i in range(1, 26)]]
  df_test = test[[f'{i}_{cat}' for i in range(1, 26)]]
  mean_train = df_train.mean(axis = 1)
  mean_test = df_test.mean(axis = 1)
  
  return mean_train, mean_test


In [ ]:
# mean_train, mean_test = get_combined_metrics_for_tiles('elevation', train_new, test_new)
# 

In [ ]:
# df_train_averaged = df

Encode elevation as categorical variable

In [ ]:
# Change Elevation to categorical

df_elevation_train = train[[f'{i}_elevation' for i in range(1, 26)]]

df_elevation_train[df_elevation_train < 101] = 1
df_elevation_train[list(101 >= df_elevation_train) and list(df_elevation_train < 192)] = 2
df_elevation_train[list(192 >= df_elevation_train) and list(df_elevation_train < 312)] = 3
df_elevation_train[list(312 >= df_elevation_train)] = 4


df_elevation_test = test[[f'{i}_elevation' for i in range(1, 26)]]

df_elevation_test[df_elevation_test < 101] = 1
df_elevation_test[list(101 >= df_elevation_test) and list(df_elevation_test < 192)] = 2
df_elevation_test[list(192 >= df_elevation_test) and list(df_elevation_test < 312)] = 3
df_elevation_test[list(312 >= df_elevation_test)] = 4

# for i in range(1,26):
#   df_elevation[i] = np.where(df_elevation[i].between(101,192), 2, df_elevation[i])
#   df_elevation[i] = np.where(df_elevation[i].between(193,312), 3, df_elevation[i])
#   df_elevation[df_elevation >= 312] = 4

In [ ]:
train_new, test_new = add_df_to_train_and_test(df_elevation_train, df_elevation_test, train, test, 'catgoricalelevation')

In [ ]:

def multiply_two_cat(cat_1, cat_2, orig_df):
  df_1 = orig_df[[f'{i}_{cat_1}' for i in range(1, 26)]]
  df_2 = orig_df[[f'{i}_{cat_2}' for i in range(1, 26)]]
  # df_2[df_2 < 40] = 0
  df_2.columns = list(range(1, 26))
  df_1.columns = list(range(1, 26))
  return df_1.mul(df_2)

def apply_lambda_to_cat(f, cat, orig_df):
  df = orig_df[[f'{i}_{cat}' for i in range(1, 26)]]
  return df.apply(f)

def apply_lambda_to_train_and_test(f, cat, train, test, name):
  train_ = apply_lambda_to_cat(f, cat, train)
  test_ = apply_lambda_to_cat(f, cat, test)
  return add_df_to_train_and_test(train_, test_, train, test, name)




add tan and tanh transformations to slope

In [ ]:
geo_dict = {
    1: 0.1,
    2: 0.2,
    3: 1,
    4: 0,
    5: 0.3,
    6: 0,
    7: 0.1,
}

# def clean_outliers(array, min, max):
#   mn = np.mean(array)
#   for x in array:
#     if x > max or x < min:
#       x = mn
#   return np.array(array)


train_new, test_new = apply_lambda_to_train_and_test(lambda x: [geo_dict[i] for i in x], 'geology', train_new, test_new, 'geo_rescored')

train_new, test_new = apply_lambda_to_train_and_test(lambda x: [np.mean(x) if (i > 312.0 or i < 101.75) else i for i in x], 'elevation', train_new, test_new, 'elevationnoout')

train_new, test_new = apply_lambda_to_train_and_test(lambda x: [45.0 if (i > 50.0) else i for i in x], 'slope', train_new, test_new, 'slopenoout')

train_new, test_new = apply_lambda_to_train_and_test(lambda x: np.tan(2*np.pi/360 * x), 'slope', train_new, test_new, 'slope_tan')

train_new, test_new = apply_lambda_to_train_and_test(lambda x: np.tanh(2*np.pi/360 * x), 'slope', train_new, test_new, 'slope_tanh')

train_new, test_new = apply_lambda_to_train_and_test(lambda x: np.log(x), 'slope', train_new, test_new, 'slope_log')


In [ ]:
# slope_tan_train

In [ ]:
# train_ = multiply_two_cat('geology', 'slope', train)
# test_ = multiply_two_cat('geology', 'slope', test)
# train_new, test_new = add_df_to_train_and_test(train_, test_, train, test, 'geovsslope')

# train_ = multiply_two_cat('twi', 'slope', train)
# test_ = multiply_two_cat('twi', 'slope', test)
# train_new, test_new = add_df_to_train_and_test(train_, test_, train, test, '-twi-vs-slope')

# train_new, test_new = train_new, test_new


In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = StandardScaler()
# scaler = RobustScaler()

new_df = train_new[train_new.columns[1:-1]].copy(deep=True)
new_df = pd.DataFrame(scaler.fit_transform(new_df))
new_df.columns = train_new.columns[1:-1]
new_df['Sample_ID'] = train_new['Sample_ID']
new_df = new_df[[new_df.columns[-1]]+list(new_df.columns[:-1])]
new_df['Label'] = train_new['Label']
train_new = new_df 
test_new[train_new.columns[1:-1]] = pd.DataFrame(scaler.transform(test_new[train_new.columns[1:-1]]))

In [ ]:
def block_col(train, test, col):
  new_train = train[filter(lambda x: col not in x, train.columns)]
  new_test = test[filter(lambda x: col not in x, test.columns)]
  return new_train, new_test

In [ ]:
# train_new, test_new = block_col(train_new, test_new, '_')
# train_new, test_new = block_col(train_new, test_new, '_elevation')
# train_new, test_new = block_col(train_new, test_new, '_aspect')

<a name = "Model"></a>
## 9. Feature Selection

In [ ]:
# # Select main columns to be used in training
# main_cols = train_new.columns.difference(['Sample_ID', 'Label'])
# X = train_new[main_cols]
# y = train_new.Label

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=2022)

# number_cat = int((len(test_new.columns)-1)/25)
# existing_cols = []
# best_feats = []
# best_scores = []
# categories_as_num = list(range(number_cat))
# for feat in range(number_cat):
#   best_feat = 0
#   best_score = 0
  
#   best_cols = []
#   for i in categories_as_num:
#     # if i == 9:
#     #   new_cols_i = list(train_new.columns[9*25+1:-1])
#     #   cols_i = new_cols_i + existing_cols
#     # else:
#     new_cols_i = list(train_new.columns[i*25+1:(i+1)*25+1])
#     # print(new_cols_i)

#     cols_i = new_cols_i + existing_cols
#     X_i = train_new[cols_i]
#     y_i = train_new.Label
    

#     # Split data into train and test sets
#     X_i_train, X_i_test, y_i_train, y_i_test = train_test_split(X_i,y_i,test_size=0.3, random_state=2022)

#     # Train model
#     model = RandomForestClassifier(random_state = 2022)
#     model.fit(X_i_train, y_i_train)

#     # Make predictions
#     y_i_pred = model.predict(X_i_test)

#     # Check the auc score of the model
#     # print(f'RandomForest F1 score on the X_test is: {f1_score(y_i_test, y_i_pred)}\n')
#     if f1_score(y_i_test, y_i_pred) > best_score:
#       name = new_cols_i[0].split('_')[1] 
#       best_score = f1_score(y_i_test, y_i_pred)
#       best_feat = i
#       best_cols = new_cols_i

#     # print classification report
#     # print(classification_report(y_i_test, y_i_pred))
#   existing_cols = existing_cols + best_cols
#   best_feats.append(best_feat)
#   best_scores.append(best_score)
#   categories_as_num.remove(best_feat)
#   print(f'{best_feat} : {name}: {best_score}')


In [ ]:
# plt.plot(best_scores)

In [ ]:
# existing_cols = ['1_slope', '2_slope', '3_slope', '4_slope', '5_slope', '6_slope', '7_slope', '8_slope', '9_slope', '10_slope', '11_slope', '12_slope', '13_slope', '14_slope', '15_slope', '16_slope', '17_slope', '18_slope', '19_slope', '20_slope', '21_slope', '22_slope', '23_slope', '24_slope', '25_slope', '1_geology', '2_geology', '3_geology', '4_geology', '5_geology', '6_geology', '7_geology', '8_geology', '9_geology', '10_geology', '11_geology', '12_geology', '13_geology', '14_geology', '15_geology', '16_geology', '17_geology', '18_geology', '19_geology', '20_geology', '21_geology', '22_geology', '23_geology', '24_geology', '25_geology', '1_elevation', '2_elevation', '3_elevation', '4_elevation', '5_elevation', '6_elevation', '7_elevation', '8_elevation', '9_elevation', '10_elevation', '11_elevation', '12_elevation', '13_elevation', '14_elevation', '15_elevation', '16_elevation', '17_elevation', '18_elevation', '19_elevation', '20_elevation', '21_elevation', '22_elevation', '23_elevation', '24_elevation', '25_elevation', '1_sdoif', '2_sdoif', '3_sdoif', '4_sdoif', '5_sdoif', '6_sdoif', '7_sdoif', '8_sdoif', '9_sdoif', '10_sdoif', '11_sdoif', '12_sdoif', '13_sdoif', '14_sdoif', '15_sdoif', '16_sdoif', '17_sdoif', '18_sdoif', '19_sdoif', '20_sdoif', '21_sdoif', '22_sdoif', '23_sdoif', '24_sdoif', '25_sdoif', '1_lsfactor', '2_lsfactor', '3_lsfactor', '4_lsfactor', '5_lsfactor', '6_lsfactor', '7_lsfactor', '8_lsfactor', '9_lsfactor', '10_lsfactor', '11_lsfactor', '12_lsfactor', '13_lsfactor', '14_lsfactor', '15_lsfactor', '16_lsfactor', '17_lsfactor', '18_lsfactor', '19_lsfactor', '20_lsfactor', '21_lsfactor', '22_lsfactor', '23_lsfactor', '24_lsfactor', '25_lsfactor', '1_placurv', '2_placurv', '3_placurv', '4_placurv', '5_placurv', '6_placurv', '7_placurv', '8_placurv', '9_placurv', '10_placurv', '11_placurv', '12_placurv', '13_placurv', '14_placurv', '15_placurv', '16_placurv', '17_placurv', '18_placurv', '19_placurv', '20_placurv', '21_placurv', '22_placurv', '23_placurv', '24_placurv', '25_placurv', '1_twi', '2_twi', '3_twi', '4_twi', '5_twi', '6_twi', '7_twi', '8_twi', '9_twi', '10_twi', '11_twi', '12_twi', '13_twi', '14_twi', '15_twi', '16_twi', '17_twi', '18_twi', '19_twi', '20_twi', '21_twi', '22_twi', '23_twi', '24_twi', '25_twi', '1_procurv', '2_procurv', '3_procurv', '4_procurv', '5_procurv', '6_procurv', '7_procurv', '8_procurv', '9_procurv', '10_procurv', '11_procurv', '12_procurv', '13_procurv', '14_procurv', '15_procurv', '16_procurv', '17_procurv', '18_procurv', '19_procurv', '20_procurv', '21_procurv', '22_procurv', '23_procurv', '24_procurv', '25_procurv', '1_aspect', '2_aspect', '3_aspect', '4_aspect', '5_aspect', '6_aspect', '7_aspect', '8_aspect', '9_aspect', '10_aspect', '11_aspect', '12_aspect', '13_aspect', '14_aspect', '15_aspect', '16_aspect', '17_aspect', '18_aspect', '19_aspect', '20_aspect', '21_aspect', '22_aspect', '23_aspect', '24_aspect', '25_aspect', '1_geo_cat_1', '2_geo_cat_1', '3_geo_cat_1', '4_geo_cat_1', '5_geo_cat_1', '6_geo_cat_1', '7_geo_cat_1', '8_geo_cat_1', '9_geo_cat_1', '10_geo_cat_1', '11_geo_cat_1', '12_geo_cat_1', '13_geo_cat_1', '14_geo_cat_1', '15_geo_cat_1', '16_geo_cat_1', '17_geo_cat_1', '18_geo_cat_1', '19_geo_cat_1', '20_geo_cat_1', '21_geo_cat_1', '22_geo_cat_1', '23_geo_cat_1', '24_geo_cat_1', '25_geo_cat_1', '1_geo_cat_2', '2_geo_cat_2', '3_geo_cat_2', '4_geo_cat_2', '5_geo_cat_2', '6_geo_cat_2', '7_geo_cat_2', '8_geo_cat_2', '9_geo_cat_2', '10_geo_cat_2', '11_geo_cat_2', '12_geo_cat_2', '13_geo_cat_2', '14_geo_cat_2', '15_geo_cat_2', '16_geo_cat_2', '17_geo_cat_2', '18_geo_cat_2', '19_geo_cat_2', '20_geo_cat_2', '21_geo_cat_2', '22_geo_cat_2', '23_geo_cat_2', '24_geo_cat_2', '25_geo_cat_2', '1_geo_cat_3', '2_geo_cat_3', '3_geo_cat_3', '4_geo_cat_3', '5_geo_cat_3', '6_geo_cat_3', '7_geo_cat_3', '8_geo_cat_3', '9_geo_cat_3', '10_geo_cat_3', '11_geo_cat_3', '12_geo_cat_3', '13_geo_cat_3', '14_geo_cat_3', '15_geo_cat_3', '16_geo_cat_3', '17_geo_cat_3', '18_geo_cat_3', '19_geo_cat_3', '20_geo_cat_3', '21_geo_cat_3', '22_geo_cat_3', '23_geo_cat_3', '24_geo_cat_3', '25_geo_cat_3', '1_geo_cat_4', '2_geo_cat_4', '3_geo_cat_4', '4_geo_cat_4', '5_geo_cat_4', '6_geo_cat_4', '7_geo_cat_4', '8_geo_cat_4', '9_geo_cat_4', '10_geo_cat_4', '11_geo_cat_4', '12_geo_cat_4', '13_geo_cat_4', '14_geo_cat_4', '15_geo_cat_4', '16_geo_cat_4', '17_geo_cat_4', '18_geo_cat_4', '19_geo_cat_4', '20_geo_cat_4', '21_geo_cat_4', '22_geo_cat_4', '23_geo_cat_4', '24_geo_cat_4', '25_geo_cat_4', '1_geo_cat_5', '2_geo_cat_5', '3_geo_cat_5', '4_geo_cat_5', '5_geo_cat_5', '6_geo_cat_5', '7_geo_cat_5', '8_geo_cat_5', '9_geo_cat_5', '10_geo_cat_5', '11_geo_cat_5', '12_geo_cat_5', '13_geo_cat_5', '14_geo_cat_5', '15_geo_cat_5', '16_geo_cat_5', '17_geo_cat_5', '18_geo_cat_5', '19_geo_cat_5', '20_geo_cat_5', '21_geo_cat_5', '22_geo_cat_5', '23_geo_cat_5', '24_geo_cat_5', '25_geo_cat_5', '1_geo_cat_6', '2_geo_cat_6', '3_geo_cat_6', '4_geo_cat_6', '5_geo_cat_6', '6_geo_cat_6', '7_geo_cat_6', '8_geo_cat_6', '9_geo_cat_6', '10_geo_cat_6', '11_geo_cat_6', '12_geo_cat_6', '13_geo_cat_6', '14_geo_cat_6', '15_geo_cat_6', '16_geo_cat_6', '17_geo_cat_6', '18_geo_cat_6', '19_geo_cat_6', '20_geo_cat_6', '21_geo_cat_6', '22_geo_cat_6', '23_geo_cat_6', '24_geo_cat_6', '25_geo_cat_6', '1_geo_cat_7', '2_geo_cat_7', '3_geo_cat_7', '4_geo_cat_7', '5_geo_cat_7', '6_geo_cat_7', '7_geo_cat_7', '8_geo_cat_7', '9_geo_cat_7', '10_geo_cat_7', '11_geo_cat_7', '12_geo_cat_7', '13_geo_cat_7', '14_geo_cat_7', '15_geo_cat_7', '16_geo_cat_7', '17_geo_cat_7', '18_geo_cat_7', '19_geo_cat_7', '20_geo_cat_7', '21_geo_cat_7', '22_geo_cat_7', '23_geo_cat_7', '24_geo_cat_7', '25_geo_cat_7']
def create_list_with_cats(cat_list):
  return_list = []
  for element in cat_list:
    return_list += [f'{i}_{element}' for i in range(1, 26)]
  return return_list
cats = ['slopenoout', 'geology', 'elevation', 'sdoif', 'twi', 'aspect', 'placurv']
existing_cols = create_list_with_cats(cats)

In [ ]:
from sklearn.model_selection import KFold

n = len(cats)

train_new = train_new[['Sample_ID']+existing_cols+['Label']]

test_new = test_new[['Sample_ID']+existing_cols]

main_cols = train_new.columns.difference(['Sample_ID', 'Label'])

X = train_new[main_cols]
y = train_new.Label



# from imblearn.under_sampling import RandomUnderSampler

# ros = RandomUnderSampler(random_state=0)
# ros.fit(X, y)
# X, y = ros.fit_resample(X, y)



In [ ]:
# Split data into train and test sets
# kf = KFold(n_splits=2, random_state = 2022, shuffle=True)
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=2022)

In [ ]:
total = len(train_new['Label'])
pos = sum(train_new['Label'])
neg = total - pos
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

# weight_for_0 = 1
# weight_for_1 = 1

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))


In [ ]:
def calculate_f1(precision, recall):
  return 2 * (precision * recall)/ (precision + recall)


In [ ]:
import tensorflow as tf

from tensorflow.keras import layers

shape = n*25

def simple_nn_model(input_shape):
  inputs = layers.Input(shape=input_shape)
  x = layers.Dense(30, activation='relu')(inputs)
  x = layers.Dropout(0.4)(x)
  x = layers.Dense(20, activation='relu')(x)
  x = layers.Dropout(0.4)(x)
  x = layers.Dense(10, activation='relu')(x)
  x = layers.Dropout(0.4)(x)
  # x = layers.Dense(run['layer_sizes'][2], activation='relu')(x)
  # x = layers.Dropout(run['dropout_factor'])(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  print(model.summary())
  return model


# def conv_nn_model(input_shape=225):
#   inputs = layers.Input(shape=input_shape)

#   intermediate = tf.keras.layers.Reshape((225,1), input_shape=(225,))(inputs)

#   first_third = tf.keras.layers.Cropping1D(cropping=(0,150))(intermediate)
#   first_third = tf.keras.layers.Reshape((75,), input_shape=(75,1))(first_third)

#   # second_half = tf.keras.layers.Cropping1D(cropping=(300,0))(intermediate)
#   # second_half = tf.keras.layers.Reshape((200,), input_shape=(200,1))(second_half)
#   x = layers.Reshape((5, 5, 3))(first_third)
#   resnet = tf.keras.applications.resnet50.ResNet50(
#       include_top=False, weights='imagenet', input_tensor=inputs,
#       input_shape=(5, 5, 3), pooling='avg')
#   for i in resnet.layers:
#       i.trainable = False
#   x = resnet(x)
#   x = layers.Dense(20, activation='relu')(inputs)
#   x = layers.Dropout(0)(x)

#   outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
#   model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

#   print(model.summary())

#   return model

def conv_nn_model(input_shape):
  inputs = layers.Input(shape=input_shape)

  x = layers.Reshape((5, 5, n))(inputs)

  x = layers.Conv2D(32, (2, 2), strides = (1,1), input_shape=(5, 5, n))(x)
  x = layers.MaxPooling2D((2, 2))(x)
  # x = layers.Conv2D(64, (3, 3), activation='relu')(x)
  # x = layers.MaxPooling2D((2, 2))(x)
  # x = layers.Conv2D(64, (3, 3), activation='relu')(x)


  x = layers.Flatten()(x)
  x = layers.Dense(16, activation='relu')(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(8, activation='relu')(x)
  x = layers.Dropout(0.3)(x)

  outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

  print(model.summary())

  return model

def avg_nn_model(input_shape):
  inputs = layers.Input(shape=input_shape)

  x = layers.Reshape((5, 5, n))(inputs)
  avg_x = tf.keras.layers.Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2]))(x)
  max_x = tf.keras.layers.Lambda(lambda x: tf.reduce_max(x, axis=[1, 2]))(x)
  min_x = tf.keras.layers.Lambda(lambda x: tf.reduce_min(x, axis=[1, 2]))(x)
  # diff_x = tf.keras.layers.Subtract()([max_x, min_x])
  std_x = tf.keras.layers.Lambda(lambda x: tf.math.reduce_std(x, axis=[1, 2]))(x)

  x = tf.keras.layers.Concatenate(axis=1)([avg_x, std_x, max_x, min_x])

  # x = tf.reduce_mean(x, axis=[1, 2])

  # x = layers.Conv2D(32, (2, 2), strides = (1,1), input_shape=(5, 5, n))(x)
  # x = layers.MaxPooling2D((2, 2))(x)
  # x = layers.Conv2D(64, (3, 3), activation='relu')(x)
  # x = layers.MaxPooling2D((2, 2))(x)
  # x = layers.Conv2D(64, (3, 3), activation='relu')(x)


  # x = layers.Flatten()(x)
  x = layers.Dense(16, activation='relu')(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Dense(8, activation='relu')(x)
  x = layers.Dropout(0.3)(x)

  outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

  print(model.summary())

  return model




# my_model.fit(X_train, y_train)

model = avg_nn_model(shape)

In [ ]:
# X = np.array(X)
# y = np.array(y)
for model_i in [conv_nn_model]:
  # f1_scores = []
  # for train_index, test_index in kf.split(X):
  #   # print("TRAIN:", train_index, "TEST:", test_index)
  #   X_train, X_test = X[train_index], X[test_index]
  #   y_train, y_test = y[train_index], y[test_index]
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=2022)
  
  model = model_i(shape)

  loss = tf.keras.losses.binary_crossentropy

  optimizer = tf.keras.optimizers.Adam()

  metrics = [tf.keras.metrics.BinaryAccuracy(name='test_accuracy'),
              tf.keras.metrics.Recall(name='recall'),
              tf.keras.metrics.Precision(name='precision'),
              tf.keras.metrics.FalseNegatives(name='fn'),
              tf.keras.metrics.FalsePositives(name='fp'),
              tf.keras.metrics.TrueNegatives(name='tn'),
              tf.keras.metrics.TruePositives(name='tp')]

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics)
  
  # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=1e-3,
  #                                                           restore_best_weights=True)

  history_simple = model.fit(X_train, y_train, validation_data = [X_test, y_test], epochs=20, 
                            verbose=2, class_weight=class_weight)
  
  f1_score = calculate_f1(np.array(history_simple.history['val_precision']), np.array(history_simple.history['val_recall']))[-1]

  # f1_scores.append(f1_score)
  print(f1_score)
  # print(f'mean: {np.mean(f1_scores)}')

In [ ]:
from xgboost import XGBRegressor, XGBClassifier

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=2022)
xgboost_model = XGBRegressor(n_estimators=500, learning_rate=0.05)
xgboost_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False)
# xgboost_model = lambda x: np.sigmoid(xgboost_model(x))

In [ ]:
# xgboost_model = XGBClassificator(n_estimators=500, learning_rate=0.05)
# xgboost_model.fit(X_train, y_train, 
#              )
xgb_cl = XGBClassifier(class_weight='balanced')

# Fit
xgb_cl.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False)

In [ ]:
rfc_model = RandomForestClassifier(n_estimators = 100, random_state = 2022, class_weight='balanced')
rfc_model.fit(X_train, y_train)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=20)
knn_model.fit(X_train, y_train)

In [ ]:
from sklearn.svm import SVC
svc_model = SVC(probability=True, kernel='rbf', class_weight='balanced')
svc_model.fit(X_train, y_train)

In [ ]:
  # X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=2022)
avg_model = avg_nn_model(shape)

loss = tf.keras.losses.binary_crossentropy

optimizer = tf.keras.optimizers.Adam()

metrics = [tf.keras.metrics.BinaryAccuracy(name='test_accuracy'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.FalseNegatives(name='fn'),
            tf.keras.metrics.FalsePositives(name='fp'),
            tf.keras.metrics.TrueNegatives(name='tn'),
            tf.keras.metrics.TruePositives(name='tp')]

avg_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=1e-3,
#                                                           restore_best_weights=True)

history_avg = avg_model.fit(X_train, y_train, validation_data = [X_test, y_test], epochs=20, 
                          verbose=2, class_weight=class_weight)

f1_score = calculate_f1(np.array(history_avg.history['val_precision']), np.array(history_simple.history['val_recall']))[-1]

# f1_scores.append(f1_score)
print(f1_score)

lr -> 0.001

In [ ]:
def plot_hist_f1(history):
  hist_f1 = calculate_f1(np.array(history.history['precision']), np.array(history.history['recall']))
  hist_f1_val = calculate_f1(np.array(history.history['val_precision']), np.array(history.history['val_recall']))
  plt.plot(hist_f1, label = 'train')
  plt.plot(hist_f1_val, label ='test')
  plt.xlabel('epochs')
  plt.ylabel('f1_score')
  plt.legend()
  plt.show()
def calculate_f1(precision, recall):
  return 2 * (precision * recall)/ (precision + recall)
plot_hist_f1(history_simple)



# print(history.history['precision'] .* history.history['recall'])

In [ ]:
def combined_predictions(model_predictions, weights):
  predictions = np.zeros((len(model_predictions['AVG'].flatten(),)))
  print(len(predictions))

  for weight, prediction in zip(weights, model_predictions.values()):
    predictions = predictions + weight * np.array(prediction.flatten())
  return predictions


  

In [ ]:
xgboost_model.predict(X_test)


In [ ]:
sigmoid = lambda x: 1/(1 + np.exp(-x))

In [ ]:
# Confusion matrix
model_predictions = {'Regr': xgboost_model.predict(X_test), 
                     'Conv': model.predict(X_test), 
                     'AVG': avg_model.predict(X_test), 
                     'cli': xgb_cl.predict_proba(X_test)[:, 1],
                     'rfc': rfc_model.predict_proba(X_test)[:, 1],
                     'gnb': gnb_model.predict_proba(X_test)[:, 1],
                     'knn': knn_model.predict_proba(X_test)[:, 1],
                     'svc': svc_model.predict_proba(X_test)[:, 1]}
weights = [0.3, 0.05, 0.05, 0.05, 0.3, 0.05, 0.05, 0.15]
thres = 0.38
# y_pred_xg = xgboost_model.predict(X_test)
# y_pred = model.predict(X_test)
# y_pred_avg = avg_model.predict(X_test)
# y_pred_res = np.mean([np.array(y_pred_xg.flatten()), np.array(y_pred.flatten()), np.array(y_pred_avg.flatten())], axis=0)
y_pred_res = combined_predictions(model_predictions, weights)

cm = confusion_matrix(y_test, [1 if x >= thres else 0 for x in y_pred_res.flatten() ])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(15,7))
disp.plot(ax=ax)
plt.show()

In [ ]:
print(y_pred_res)

In [ ]:
# y_pred_xg

In [ ]:
# Feature importance
# impo_df = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_}).set_index('feature').sort_values(by = 'importance', ascending = False)
# impo_df = impo_df[:10].sort_values(by = 'importance', ascending = True)
# impo_df.plot(kind = 'barh', figsize = (10, 10))
# plt.legend(loc = 'center right')
# plt.title('Bar chart showing top ten features', fontsize = 14)
# plt.xlabel('Features', fontsize = 12, color = 'indigo')
# plt.show()

<a name = "Predictions"></a>
## 10. Test set predictions

In [ ]:
# Make prediction on the test set
# main_cols = test_new.columns[1:]

test_df = test_new[X_test.columns]
model_predictions = {'Regr': xgboost_model.predict(test_df), 
                     'Conv': model.predict(test_df), 
                     'AVG': avg_model.predict(test_df), 
                     'cli': xgb_cl.predict_proba(test_df)[:, 1],
                     'rfc': rfc_model.predict_proba(test_df)[:, 1],
                     'gnb': gnb_model.predict_proba(test_df)[:, 1],
                     'knn': knn_model.predict_proba(test_df)[:, 1],
                     'svc': svc_model.predict_proba(test_df)[:, 1]}
# y_pred_xg = xgboost_model.predict(test_df)
# y_pred = model.predict(test_df)
# y_pred_avg = avg_model.predict(test_df)
# y_pred_res = np.mean([np.array(y_pred_xg.flatten()), np.array(y_pred.flatten()), np.array(y_pred_avg.flatten())], axis=0)
y_pred_res = combined_predictions(model_predictions, weights)
# Create a submission file
sub_file = pd.DataFrame({'Sample_ID': test_new.Sample_ID, 'Label': [1 if x >= thres else 0 for x in y_pred_res.flatten() ]})

# Check the distribution of your predictions
sns.countplot(x = sub_file.Label)
plt.title('Predicted Variable Distribution');

<a name = "Submission"></a>
## 11. Creating a submission file

In [ ]:
# Create a csv file and upload to zindi 
sub_file.to_csv('submission.csv', index = False)
sub_file

<a name = "Tips"></a>
## 12. Tips to improve model performance
 - Use cross-validation techniques
 - Feature engineering
 - Handle the class imbalance of the target variable
 - Try different modelling techniques - Stacking classifier, Voting classifiers, ensembling...
 - Data transformations
 - Feature Selection techniques such as RFE, Tree-based feature importance...
 - Domain Knowledge, do research on how the provided features affect landslides, soil topology...

#                       ::GOOD LUCK AND HAPPY HACKING 😊


